In [1]:
import pandas as pd
from pulp import *
import re
from scipy.stats import binom
import numpy as np

In [2]:
data = pd.read_csv("./data/IPL-aggregated-stats_final.csv")

In [3]:
data.head()

,PLAYER,Pts_sum,Pts_mean,years_played,Mat_sum,Mat_mean,Wkts_sum,Wkts_mean,Dots_sum,Dots_mean,...,6s_mean,Catches_sum,Catches_mean,Stumpings_sum,Stumpings_mean,position,overseas_or_indian,median_season_ratings,avg_season_ratings,base_price
0,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.000000,0,0.000000,...,18.181818,80,7.272727,7,0.636364,BAT,Overseas,50.344828,51.410021,1.50
1,AaronFinch,751.0,83.444444,9,75,8.333333,1,0.111111,13,1.444444,...,7.444444,24,2.666667,0,0.000000,BAT,Overseas,11.699010,27.457438,0.75
2,AbhishekJhunjhunwala,120.5,40.166667,3,21,7.000000,1,0.333333,27,9.000000,...,1.666667,10,3.333333,0,0.000000,BAT,Indian,2.472464,4.623134,0.30
3,AbhishekNayar,340.5,48.642857,7,60,8.571429,9,1.285714,69,9.857143,...,2.857143,13,1.857143,0,0.000000,BAT,Indian,2.480645,8.571047,0.30
4,AdamGilchrist,1091.5,181.916667,6,80,13.333333,1,0.166667,1,0.166667,...,15.333333,51,8.500000,16,2.666667,WK,Overseas,70.380747,61.764019,2.00


In [60]:
selected_list = ["DavidWarner", "AxarPatel", "AdamGilchrist", "NitishRana", "MohitSharma", "BhuvneshwarKumar",
                 "MitchellMcClenaghan", "AnilKumble", "AzharMahmood", "YuvrajSingh", "PraveenKumar"]
sum(data[data.PLAYER.isin(selected_list)]["median_season_ratings"])

742.3838527243583

In [47]:
data.overseas_or_indian.value_counts()

Indian      135
Overseas    104
Name: overseas_or_indian, dtype: int64

### The objective function along with other constraints are mentioned below

$$ max \sum_{players =1}^N rating$$
s.t $$ min-batsman <  chosen-batsman < max-batsman $$
s.t $$ min-batsman <  chosen-batsman < max-batsman $$
s.t $$ min-bowlers <  chosen-bowlers < max-bowlers $$
s.t $$ min-allrounders <  chosen-all-rounders < max-allrounders $$
s.t $$ wicketkeepers = 1 $$

### Creating the set of players that will go into the auction

In [4]:
players = list(data["PLAYER"])
indians = list(data[data["overseas_or_indian"]=="Indian"]["PLAYER"])
overseas = list(data[data["overseas_or_indian"]=="Overseas"]["PLAYER"])
batsman = list(data[data.position=="BAT"]["PLAYER"])
bowlers = list(data[data.position=="BWL"]["PLAYER"])
allrounders = list(data[data.position=="AR"]["PLAYER"])
wks = list(data[data.position=="WK"]["PLAYER"])

In [63]:
data[data.PLAYER.isin(["PraveenKumar", "ZaheerKhan", "SiddarthKaul"])]

,PLAYER,Pts_sum,Pts_mean,years_played,Mat_sum,Mat_mean,Wkts_sum,Wkts_mean,Dots_sum,Dots_mean,...,6s_mean,Catches_sum,Catches_mean,Stumpings_sum,Stumpings_mean,position,overseas_or_indian,median_season_ratings,avg_season_ratings,base_price
160,PraveenKumar,1538.0,153.80,10,119,11.9,90,9.0,1076,107.6,...,1.7,12,1.2,0,0.0,BWL,Indian,70.410509,53.318622,2.00
205,SiddarthKaul,509.0,101.80,5,45,9.0,49,9.8,320,64.0,...,0.0,7,1.4,0,0.0,BWL,Indian,8.929870,28.434179,0.75
238,ZaheerKhan,1321.5,132.15,10,100,10.0,102,10.2,875,87.5,...,0.2,22,2.2,0,0.0,BWL,Indian,39.205645,42.608871,1.00


In [49]:
median_ratings = dict(zip(players, list(data["median_season_ratings"])))
base_prices = dict(zip(players, list(data["base_price"])))

### Lets implement a Binary Integer Programming strategy

In [52]:
players_lp = LpVariable.dicts("Chosen",players,0,1,cat='Integer')

In [53]:
def filter_lp_dicts(players_lp, criteria_list):
    filter_dict = {}
    for player in criteria_list:
        filter_dict[player] = players_lp[player]
    return filter_dict

In [54]:
batsman_lp = filter_lp_dicts(players_lp, batsman)
bowlers_lp = filter_lp_dicts(players_lp, bowlers)
ar_lp = filter_lp_dicts(players_lp, allrounders)
wk_lp = filter_lp_dicts(players_lp, wks)
indians_lp = filter_lp_dicts(players_lp, indians)
overseas_lp = filter_lp_dicts(players_lp, overseas)

In [55]:
def constructLpSumArr(dict_lp):
    lp_arr = []
    for k,v in dict_lp.items():
        lp_arr += lpSum(dict_lp[k])
    return lp_arr

In [56]:
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in players_lp.items():
    costs += lpSum(base_prices[k] * players_lp[k])
    rewards += lpSum(median_ratings[k] * players_lp[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(constructLpSumArr(wk_lp)) == 1
prob += lpSum(constructLpSumArr(players_lp)) == 11
prob += lpSum(constructLpSumArr(batsman_lp)) >= 3
prob += lpSum(constructLpSumArr(batsman_lp)) <= 5
prob += lpSum(constructLpSumArr(bowlers_lp)) >= 3
prob += lpSum(constructLpSumArr(bowlers_lp))<= 5
prob += lpSum(constructLpSumArr(ar_lp)) >= 1
prob += lpSum(constructLpSumArr(ar_lp)) <= 3
prob += lpSum(constructLpSumArr(indians_lp)) == 7
prob += lpSum(constructLpSumArr(overseas_lp)) == 4

In [57]:
prob.solve()

1

In [111]:
prob.writeLP("./models/IPL_Best11_BasePrice.lp")

[Chosen_ABdeVilliers,
 Chosen_AaronFinch,
 Chosen_AbhishekJhunjhunwala,
 Chosen_AbhishekNayar,
 Chosen_AdamGilchrist,
 Chosen_AjanthaMendis,
 Chosen_AjinkyaRahane,
 Chosen_AjitAgarkar,
 Chosen_AkshdeepNath,
 Chosen_AlbieMorkel,
 Chosen_AmbatiRayudu,
 Chosen_AmitMishra,
 Chosen_AndreRussell,
 Chosen_AndrewSymonds,
 Chosen_AndrewTye,
 Chosen_AngeloMathews,
 Chosen_AnilKumble,
 Chosen_AnkitRajpoot,
 Chosen_AnkitSharma,
 Chosen_AnureetSingh,
 Chosen_AshishNehra,
 Chosen_AshishReddy,
 Chosen_AshokDinda,
 Chosen_AxarPatel,
 Chosen_AzharMahmood,
 Chosen_BarinderSran,
 Chosen_BenCutting,
 Chosen_BenStokes,
 Chosen_BharatChipli,
 Chosen_BhuvneshwarKumar,
 Chosen_BipulSharma,
 Chosen_BradHodge,
 Chosen_BradHogg,
 Chosen_BrendonMcCullum,
 Chosen_BrettLee,
 Chosen_CMGautam,
 Chosen_CameronWhite,
 Chosen_ChamindaVaas,
 Chosen_CharlLangeveldt,
 Chosen_CheteshwarPujara,
 Chosen_ChrisGayle,
 Chosen_ChrisJordan,
 Chosen_ChrisLynn,
 Chosen_ChrisMorris,
 Chosen_ColinMunro,
 Chosen_ColindeGrandhomme,
 Cho

In [59]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [60]:
summary(prob)

Variables:

Chosen_AnilKumble = 1.0
Chosen_BhuvneshwarKumar = 1.0
Chosen_ChrisGayle = 1.0
Chosen_DavidWarner = 1.0
Chosen_HarbhajanSingh = 1.0
Chosen_JacquesKallis = 1.0
Chosen_MunafPatel = 1.0
Chosen_RashidKhan = 1.0
Chosen_RishabhPant = 1.0
Chosen_SureshRaina = 1.0
Chosen_VirenderSehwag = 1.0
---------------------------------------

Constraints:
2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 = 22.0
---------------------------------------

Score:
81.0*1.0 + 80.66666666666667*1.0 + 80.47619047619048*1.0 + 80.55555555555556*1.0 + 80.49107142857143*1.0 + 70.33333333333333*1.0 + 80.38461538461539*1.0 + 81.66666666666667*1.0 + 75.96428571428572*1.0 + 80.51315789473685*1.0 + 80.58823529411767*1.0 = 872.6397784147397


### Obviously players wont get sold at their base price. We need to use some inflated price to predict their final sell price.

In [86]:
def get_bidding_increment(price):
    if price <= 0.75:
        return 0.1
    elif (price >= 0.75 ) and (price < 1.5):
        return 0.2
    elif (price >= 1.5) and (price < 3.5):
        return 0.3
    elif (price >= 3.5) and (price < 5):
        return 0.5
    else:
        return 1

In [89]:
def price_after_betting(base_price, rounds):
    price = base_price
    while(rounds > 0):
        price += get_bidding_increment(price)
        price = round(price, 1)
        rounds-=1
    return price

In [118]:
inv_base_price_dict = {2: ">90", 1.5: "80-90", 1: "60-80", 0.75:"50-60", 0.5:"40-50", 0.4:"30-40", 0.3:"<30"}

In [203]:
def simulate_price(base_price, n=20):
    inv_base_price_dict = {2: ">90", 1.5: "80-90", 1: "60-80", 0.75:"50-60", 0.5:"40-50", 0.4:"30-40", 0.3:"<30"}
    prob_dist = {
    ">90": [binom.pmf(i, n, 0.4) for i in range(n+1)],
    "80-90": [binom.pmf(i, n, 0.3) for i in range(n+1)],
    "60-80": [binom.pmf(i, n, 0.2) for i in range(n+1)],
    "50-60": [binom.pmf(i, n, 0.15) for i in range(n+1)],
    "40-50": [binom.pmf(i, n, 0.15) for i in range(n+1)],
    "30-40": [binom.pmf(i, n, 0.12) for i in range(n+1)],
    "<30": [binom.pmf(i, n, 0.1) for i in range(n+1)]
    }
    prices = [price_after_betting(base_price, i) for i in range(n+1)]
    prob = prob_dist[inv_base_price_dict[base_price]]
    mean_simulated = sum([prices[i] * prob[i] for i in range(n+1)])
    return mean_simulated if mean_simulated > base_price else base_price

### We say all players atleast go for 5 rounds of betting (5 times the price was increased)

In [143]:
inflated_price = dict(zip(players, list(data["base_price"].apply(lambda x : price_after_betting(x, 5)))))

In [144]:
data.base_price = data.base_price.astype(float)

In [204]:
data["simulated_price"] = list(data["base_price"].apply(lambda x : simulate_price(x)))


In [205]:
simulated_price = dict(zip(players, data["simulated_price"]))

In [206]:
prob = LpProblem("IPL_Auction", LpMaximize)
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in players_lp.items():
    costs += lpSum(simulated_price[k] * players_lp[k])
    rewards += lpSum(median_ratings[k] * players_lp[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(constructLpSumArr(wk_lp)) == 1
prob += lpSum(constructLpSumArr(players_lp)) == 11
prob += lpSum(constructLpSumArr(batsman_lp)) >= 3
prob += lpSum(constructLpSumArr(batsman_lp)) <= 5
prob += lpSum(constructLpSumArr(bowlers_lp)) >= 3
prob += lpSum(constructLpSumArr(bowlers_lp))<= 5
prob += lpSum(constructLpSumArr(ar_lp)) >= 1
prob += lpSum(constructLpSumArr(ar_lp)) <= 3
prob += lpSum(constructLpSumArr(indians_lp)) == 7
prob += lpSum(constructLpSumArr(overseas_lp)) == 4

In [207]:
prob.solve()

1

In [208]:
summary(prob)

Variables:

Chosen_AjinkyaRahane = 1.0
Chosen_AxarPatel = 1.0
Chosen_BrendonMcCullum = 1.0
Chosen_DavidWarner = 1.0
Chosen_IshantSharma = 1.0
Chosen_MuttiahMuralitharan = 1.0
Chosen_NitishRana = 1.0
Chosen_RashidKhan = 1.0
Chosen_SachinTendulkar = 1.0
Chosen_SiddharthTrivedi = 1.0
Chosen_YuvrajSingh = 1.0
---------------------------------------

Constraints:
1.9288164114295732*1.0 + 3.3945146964406807*1.0 + 1.9288164114295732*1.0 + 5.4455109304120715*1.0 + 1.9288164114295732*1.0 + 1.9288164114295732*1.0 + 1.9288164114295732*1.0 + 5.4455109304120715*1.0 + 1.9288164114295732*1.0 + 1.9288164114295732*1.0 + 1.9288164114295732*1.0 = 29.716067848701407
---------------------------------------

Score:
50.23255813953488*1.0 + 62.71250000000001*1.0 + 44.22727272727273*1.0 + 80.55555555555556*1.0 + 50.23255813953488*1.0 + 50.20833333333334*1.0 + 44.20842379504993*1.0 + 81.66666666666667*1.0 + 41.257648953301135*1.0 + 44.227040816326536*1.0 + 44.19230769230769*1.0 = 593.7208658188835


In [116]:
prob.writeLP("./models/IPL_Best11_Rounds-5-Inflation.lp")

[Chosen_ABdeVilliers,
 Chosen_AaronFinch,
 Chosen_AbhishekJhunjhunwala,
 Chosen_AbhishekNayar,
 Chosen_AdamGilchrist,
 Chosen_AjanthaMendis,
 Chosen_AjinkyaRahane,
 Chosen_AjitAgarkar,
 Chosen_AkshdeepNath,
 Chosen_AlbieMorkel,
 Chosen_AmbatiRayudu,
 Chosen_AmitMishra,
 Chosen_AndreRussell,
 Chosen_AndrewSymonds,
 Chosen_AndrewTye,
 Chosen_AngeloMathews,
 Chosen_AnilKumble,
 Chosen_AnkitRajpoot,
 Chosen_AnkitSharma,
 Chosen_AnureetSingh,
 Chosen_AshishNehra,
 Chosen_AshishReddy,
 Chosen_AshokDinda,
 Chosen_AxarPatel,
 Chosen_AzharMahmood,
 Chosen_BarinderSran,
 Chosen_BenCutting,
 Chosen_BenStokes,
 Chosen_BharatChipli,
 Chosen_BhuvneshwarKumar,
 Chosen_BipulSharma,
 Chosen_BradHodge,
 Chosen_BradHogg,
 Chosen_BrendonMcCullum,
 Chosen_BrettLee,
 Chosen_CMGautam,
 Chosen_CameronWhite,
 Chosen_ChamindaVaas,
 Chosen_CharlLangeveldt,
 Chosen_CheteshwarPujara,
 Chosen_ChrisGayle,
 Chosen_ChrisJordan,
 Chosen_ChrisLynn,
 Chosen_ChrisMorris,
 Chosen_ColinMunro,
 Chosen_ColindeGrandhomme,
 Cho

In [121]:
np.mean([1,2,3])

2.0

In [178]:
binom.cdf(20, 20, 0.0001)

1.0